In [13]:
import requests
import pandas as pd
import time
import json

In [14]:
#https://www.imdb.com/interfaces/

There is a significant amount of data provided by IMDB. In this notebook, I parse through all of it and merge the relevant information into a single csv.

In [15]:
a = pd.read_csv('./data/title.akas.tsv.gz', sep='\t', compression='gzip')
b = pd.read_csv('./data/title.basics.tsv.gz', sep='\t', compression='gzip')
c = pd.read_csv('./data/title.crew.tsv.gz', sep='\t', compression='gzip')
d = pd.read_csv('./data/title.episode.tsv.gz', sep='\t', compression='gzip')
e = pd.read_csv('./data/title.principals.tsv.gz', sep='\t', compression='gzip')
f = pd.read_csv('./data/title.ratings.tsv.gz', sep='\t', compression='gzip')
g = pd.read_csv('./data/name.basics.tsv.gz', sep='\t', compression='gzip')

C:\Users\jg_ri\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
C:\Users\jg_ri\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [16]:
#renaming the unique id for consistency
a.rename(columns = {'titleId':'tconst'}, inplace=True)

In [17]:
a.shape

(30871184, 8)

In [18]:
d.shape

(6486450, 4)

In [19]:
#merging dataframes a and d
a = a.merge(d, on='tconst', how='left')

In [20]:
a.shape

(30871184, 11)

In [21]:
a.head()

,tconst,ordering,title,region,language,types,attributes,isOriginalTitle,parentTconst,seasonNumber,episodeNumber
0,tt0000001,1,Карменсіта,UA,\N,imdbDisplay,\N,0,NaN,NaN,NaN
1,tt0000001,2,Carmencita,DE,\N,\N,literal title,0,NaN,NaN,NaN
2,tt0000001,3,Carmencita - spanyol tánc,HU,\N,imdbDisplay,\N,0,NaN,NaN,NaN
3,tt0000001,4,Καρμενσίτα,GR,\N,imdbDisplay,\N,0,NaN,NaN,NaN
4,tt0000001,5,Карменсита,RU,\N,imdbDisplay,\N,0,NaN,NaN,NaN


In [22]:
#nulls are coded as '\N' so we need a function to replace these nulls
def slash_n(x):
    if x == '\\N':
        return None
    else:
        return x

In [23]:
#filling the nulls
a['episodeNumber'] = a['episodeNumber'].apply(slash_n)

In [24]:
#if an entry doesn't contain an episode number, then it must be a movie. 
a = a[a['episodeNumber'].isnull() == True]

In [25]:
#overwriting dataframe d to save memory
d = 'x'

In [26]:
temp = a[a['language'] == '\\N']

In [27]:
#restricting the dataset to english movies
a = a[a['language'] == 'en']

In [28]:
backup = a

In [29]:
# #loading in more data
# b = pd.read_csv('./data/title.basics.tsv.gz', sep='\t', compression='gzip')

In [30]:
b.shape

(8673945, 9)

In [31]:
#merging the new data onto our other dataset
a = a.merge(b, on='tconst', how='left')

In [32]:
a.shape

(318392, 19)

In [33]:
#overwriting b to save memory
b='x'

In [34]:
backup = a

In [35]:
# #reading in more data
# c = pd.read_csv('./data/title.crew.tsv.gz', sep='\t', compression='gzip')

In [36]:
#merging onto existing data
a = a.merge(c, on='tconst', how='left')

In [37]:
#overwriting c to save memory
c = 'x'

In [38]:
a.shape

(318392, 21)

In [39]:
backup = a

In [40]:
#reading in more data
# f = pd.read_csv('./data/title.ratings.tsv.gz', sep='\t', compression='gzip')
# g = pd.read_csv('./data/name.basics.tsv.gz', sep='\t', compression='gzip')

In [41]:
#merging onto existing data
a = backup.merge(f, on='tconst', how='left')

In [42]:
a.head()

,tconst,ordering,title,region,language,types,attributes,isOriginalTitle,parentTconst,seasonNumber,...,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,directors,writers,averageRating,numVotes
0,tt0000005,4,Blacksmith Scene,CA,en,imdbDisplay,\N,0,NaN,NaN,...,Blacksmith Scene,0,1893,\N,1,"Comedy,Short",nm0005690,\N,6.2,2451.0
1,tt0000010,6,La sortie de l'usine Lumière à Lyon,CA,en,imdbDisplay,\N,0,NaN,NaN,...,La sortie de l'usine Lumière à Lyon,0,1895,\N,1,"Documentary,Short",nm0525910,\N,6.9,6699.0
2,tt0000012,22,The Arrival of a Train,XWW,en,imdbDisplay,\N,0,NaN,NaN,...,L'arrivée d'un train à La Ciotat,0,1896,\N,1,"Documentary,Short","nm0525910,nm0525908",\N,7.5,11498.0
3,tt0000012,24,L'arrivée d'un train à La Ciotat,CA,en,imdbDisplay,\N,0,NaN,NaN,...,L'arrivée d'un train à La Ciotat,0,1896,\N,1,"Documentary,Short","nm0525910,nm0525908",\N,7.5,11498.0
4,tt0000012,27,The Arrival of a Train,XEU,en,imdbDisplay,\N,0,NaN,NaN,...,L'arrivée d'un train à La Ciotat,0,1896,\N,1,"Documentary,Short","nm0525910,nm0525908",\N,7.5,11498.0


In [43]:
#dropping unnecessary columns
a.drop(columns=['seasonNumber',
                'episodeNumber', 
                'parentTconst', 
                'isAdult',  
                'language', 
                'titleType',
                'types',
                'attributes'], inplace=True)

In [44]:
a.columns

Index(['tconst', 'ordering', 'title', 'region', 'isOriginalTitle',
       'primaryTitle', 'originalTitle', 'startYear', 'endYear',
       'runtimeMinutes', 'genres', 'directors', 'writers', 'averageRating',
       'numVotes'],
      dtype='object')

In [45]:
#getting rid of duplicate entries for films
a = a.drop_duplicates(subset=['tconst'], keep = 'first')

In [46]:
#filling nulls
a['endYear'] = a['endYear'].apply(slash_n)

In [47]:
#if a title has an end year, it is a tv series
a = a[a['endYear'].isnull() == True]

In [48]:
#reading in and merging more data
# e = pd.read_csv('./data/title.principals.tsv.gz', sep='\t', compression='gzip')
a = a.merge(e, on='tconst', how='left')

In [49]:
temp_list = []
for unique_id in a['tconst']:
    temp_list.append(unique_id)

In [50]:
e_temp = e[e['tconst'].isin(temp_list) == True]

In [51]:
e_temp.shape

(1630462, 6)

e is a massive dataframe providing details on the cast and crew of a movie, so in the following cells I break it down so that it ran better on my computer. 

In [52]:
#compiling all the people that worked on each movie into a single cell
e_temp_1 = e_temp[:500000]
e_temp_1 = e_temp_1.groupby('tconst').apply(lambda x: x.sum())
e_temp_1.drop(columns=['ordering', 'category', 'job', 'characters'], inplace=True)
e_temp_1['nconst'] = e_temp_1['nconst'].apply(lambda x: x.split('nm')).apply(lambda x: x[1:])

In [53]:
#compiling all the people that worked on each movie into a single cell
e_temp_2 = e_temp[500000:1000000]
e_temp_2 = e_temp_2.groupby('tconst').apply(lambda x: x.sum())
e_temp_2.drop(columns=['ordering', 'category', 'job', 'characters'], inplace=True)
e_temp_2['nconst'] = e_temp_2['nconst'].apply(lambda x: x.split('nm')).apply(lambda x: x[1:])

In [54]:
#compiling all the people that worked on each movie into a single cell
e_temp_3 = e_temp[1000000:]
e_temp_3 = e_temp_3.groupby('tconst').apply(lambda x: x.sum())
e_temp_3.drop(columns=['ordering', 'category', 'job', 'characters'], inplace=True)
e_temp_3['nconst'] = e_temp_3['nconst'].apply(lambda x: x.split('nm')).apply(lambda x: x[1:])

In [55]:
cast_crew = pd.concat([e_temp_1, e_temp_2, e_temp_3])
cast_crew.drop(columns=['tconst'], inplace=True)
cast_crew.reset_index(inplace=True)

In [56]:
e_temp = e_temp.groupby('tconst').apply(lambda x: x.sum())
e_temp.drop(columns=['ordering', 'job', 'characters'], inplace=True)
e_temp['nconst'] = e_temp['nconst'].apply(lambda x: x.split('nm')).apply(lambda x: x[1:])

In [57]:
# part_1 = e_temp[:500000]

# temp_df = pd.DataFrame(columns = e_temp.category.unique())
# temp_df['tconst'] = []
# temp_df.set_index('tconst', inplace=True)

# for index, row in part_1.iterrows():
#     temp_df.loc[row['tconst'], row['category']] = row['nconst']

In [58]:
# for index, row in part_1.iterrows():
#     temp_df.loc[row['tconst'], row['category']] = row['nconst']

In [59]:
# part_2 = e_temp[500000:750000]

# temp_df2 = pd.DataFrame(columns = e_temp.category.unique())
# temp_df2['tconst'] = []
# temp_df2.set_index('tconst', inplace=True)

# for index, row in part_2.iterrows():
#     temp_df2.loc[row['tconst'], row['category']] = row['nconst']

In [60]:
# part_3 = e_temp[750000:1000000]

# temp_df3 = pd.DataFrame(columns = e_temp.category.unique())
# temp_df3['tconst'] = []
# temp_df3.set_index('tconst', inplace=True)

# for index, row in part_3.iterrows():
#     temp_df3.loc[row['tconst'], row['category']] = row['nconst']

In [61]:
# part_4 = e_temp[1000000:1250000]

# temp_df4 = pd.DataFrame(columns = e_temp.category.unique())
# temp_df4['tconst'] = []
# temp_df4.set_index('tconst', inplace=True)

# for index, row in part_4.iterrows():
#     temp_df4.loc[row['tconst'], row['category']] = row['nconst']

In [62]:
# part_5 = e_temp[1250000:]

# temp_df5 = pd.DataFrame(columns = e_temp.category.unique())
# temp_df5['tconst'] = []
# temp_df5.set_index('tconst', inplace=True)

# for index, row in part_5.iterrows():
#     temp_df5.loc[row['tconst'], row['category']] = row['nconst']

In [63]:
# people_info = pd.concat([temp_df, temp_df2, temp_df3, temp_df4, temp_df5])
# people_info.reset_index(inplace=True)
# people_info = people_info.drop_duplicates(subset=['tconst'], keep='first')

In [64]:
final_df = a.merge(cast_crew, on='tconst', how='left')

In [65]:
final_df.shape

(1634416, 21)

In [66]:
final_df.to_csv('./final.csv')

In [67]:
# final_df.to_csv('./data/final.csv', index=False)

In [68]:
# test = e_temp[:1000]
# test = test.groupby('tconst').apply(lambda x: x.sum())
# test.drop(columns=['tconst', 'ordering', 'category', 'job', 'characters'], inplace=True)
# test['nconst'] = test['nconst'].apply(lambda x: x.split('nm')).apply(lambda x: x[1:])